In [55]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ot
import sys
import anndata
import scanpy as sc

In [57]:
chromatin_barcodes = pd.read_csv('./data/chromatin_barcodes.tsv', header=None)
chromatin_peaks = pd.read_csv('./data/chromatin_peaks.tsv', header=None)
chromatin_counts = scipy.io.mmread('./data/chromatin_counts.mtx')
adata = sc.AnnData(X=chromatin_counts.T)
adata.obs['barcodes'] = chromatin_barcodes[0].values
adata.var['peaks'] = chromatin_peaks[0].values
adata

# adata = anndata.read_h5ad("./data/multiome_processed.h5ad")
# print(adata)
# print(adata.X)
# print(adata.layers['counts'])
# print(adata.var['feature_types'])

AnnData object with n_obs × n_vars = 5081 × 229429
    obs: 'barcodes'
    var: 'peaks'

In [40]:
print((adata.var['feature_types'] == 'ATAC').iloc[13430])
print((adata.var['feature_types'] == 'ATAC').iloc[13431])

False
True


In [58]:
X = adata.X
print("full data")
print("Matrix Shape:", X.shape)
print("Number of Non-zero Elements:", X.nnz)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())
# print("Mean Value without zeros:", X[X != 0].mean())

full data
Matrix Shape: (5081, 229429)
Number of Non-zero Elements: 12998948
Density: 0.011150919981300042
Minimum Value: 0
Maximum Value: 21


In [41]:
adata_GEX = adata[:, adata.var['feature_types'] == 'GEX'].copy()
adata_GEX
X = adata_GEX.X
print("GEX data")
print("Matrix Shape:", X.shape)
print("Number of Non-zero Elements:", X.nnz)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

GEX data
Matrix Shape: (69249, 13431)
Number of Non-zero Elements: 76997578
Density: 0.08278567782807424
Minimum Value: 0.0
Maximum Value: 26882.379


In [46]:
adata_ATAC = adata[:, adata.var['feature_types'] == 'ATAC'].copy()
X = adata_ATAC.X
print("ATAC data")
print("Matrix Shape:", X.shape)
print("Number of Non-zero Elements:", X.nnz)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

ATAC data
Matrix Shape: (69249, 116490)
Number of Non-zero Elements: 248342179
Density: 0.030785650582849974
Minimum Value: 0.0
Maximum Value: 1.0
